<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Francisco Márquez
- Nombre de alumno 2: Santiago Haberle


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/fco-marquez/MDS7202_Free_Riders/tree/Lab_6/Lab6)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [102]:
# !pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [103]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         7456 non-null   int64  
 1   date       7456 non-null   object 
 2   city       7456 non-null   object 
 3   lat        7456 non-null   float64
 4   long       7456 non-null   float64
 5   pop        7456 non-null   int64  
 6   shop       7456 non-null   object 
 7   brand      7456 non-null   object 
 8   container  7456 non-null   object 
 9   capacity   7456 non-null   object 
 10  price      7456 non-null   float64
 11  quantity   7456 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 699.1+ KB


In [105]:
# Valores únicos en date, city, shop, brand, container y capacity
unique_dates = df["date"].nunique()
unique_cities = df["city"].unique()
unique_shops = df["shop"].unique()
unique_brands = df["brand"].unique()
unique_containers = df["container"].unique()
unique_capacities = df["capacity"].unique()
print(f"Unique dates: {unique_dates}")
print(f"Unique cities: {unique_cities}")
print(f"Unique shops: {unique_shops}")
print(f"Unique brands: {unique_brands}")
print(f"Unique containers: {unique_containers}")
print(f"Unique capacities: {unique_capacities}")

Unique dates: 84
Unique cities: ['Athens' 'Irakleion' 'Patra' 'Thessaloniki' 'Larisa']
Unique shops: ['shop_1' 'shop_2' 'shop_6' 'shop_4' 'shop_3' 'shop_5']
Unique brands: ['kinder-cola' 'adult-cola' 'orange-power' 'gazoza' 'lemon-boost']
Unique containers: ['glass' 'plastic' 'can']
Unique capacities: ['500ml' '1.5lt' '330ml']


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

#### 1. Train-Val-Test

In [106]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
set_config(transform_output="pandas")

X = df.drop(columns=["quantity"])
y = df["quantity"]

# División en train/val/test (70%/20%/30%)
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=1892)
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.125, random_state=1892)


In [107]:
X_train.columns

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price'],
      dtype='object')

#### 2. FunctionTransformer

In [108]:
from sklearn.preprocessing import FunctionTransformer

def extract_date_features(df):
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"], format="%d/%m/%y")
    df["day"] = df["date"].dt.day.astype("category")
    df["month"] = df["date"].dt.month.astype("category")
    df["year"] = df["date"].dt.year.astype("category")
    df = df.drop(columns=["date"])  # eliminamos la original
    return df

date_transformer = FunctionTransformer(extract_date_features)

#### 3. ColumnTransformer

In [109]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# Nombres de columnas
num_features = ["lat", "long", "pop", "price"]
cat_features = ["city", "shop", "brand", "container", "capacity", "day", "month", "year"]

categorical_transformer = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Definimos el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", categorical_transformer, cat_features)
    ],
    verbose_feature_names_out=False
)

#### 4. Pipeline DummyRegressor

In [110]:
from sklearn.dummy import DummyRegressor

pipe_dummy = Pipeline([
    ("date_features", date_transformer),
    ("preprocessing", preprocessor),
    ("model", DummyRegressor(strategy="mean"))
])

#### 5. MAE DummyRegressor

In [111]:
from sklearn.metrics import mean_absolute_error

pipe_dummy.fit(X_train, y_train)
y_pred_val = pipe_dummy.predict(X_val)

mae_dummy = mean_absolute_error(y_val, y_pred_val)
print(f"MAE DummyRegressor: {mae_dummy:.3f}")

MAE DummyRegressor: 13048.151


### Interpretación del MAE en contexto de negocios

El MAE (Mean Absolute Error) de 13048.151 indica que, en promedio, las predicciones del modelo baseline se desvían aproximadamente 13,048 unidades de la demanda real. Esto significa que si la demanda real de un producto es de 20,000 unidades, el modelo podría predecir entre 6,952 y 33,048 unidades. Este error tan alto representa un problema significativo para la planificación de inventario y producción de la empresa, ya que podría llevar a pérdidas por sobreproducción (exceso de inventario) o subproducción (pérdida de ventas por falta de stock).

#### 6. Pipeline XGBRegressor

In [112]:
from xgboost import XGBRegressor

pipe_xgb = Pipeline([
    ("date_features", date_transformer),
    ("preprocessing", preprocessor),
    ("model", XGBRegressor(random_state=1892))
])

pipe_xgb.fit(X_train, y_train)
y_pred_val_xgb = pipe_xgb.predict(X_val)

mae_xgb = mean_absolute_error(y_val, y_pred_val_xgb)
print(f"MAE XGBRegressor: {mae_xgb:.3f}")

MAE XGBRegressor: 2464.082


### Comparación XGBoost vs DummyRegressor

El modelo XGBRegressor obtiene un MAE de 2464.082, lo cual representa una reducción drástica con respecto al DummyRegressor (MAE = 13048.151). Esta mejora drástica demuestra que XGBoost es significativamente mejor que el baseline, ya que captura patrones complejos y relaciones no lineales entre las variables predictoras (precio, ubicación, fecha, tipo de producto) y la demanda. El modelo baseline solo predice el promedio histórico, mientras que XGBoost aprende interacciones entre características y ajusta predicciones según el contexto específico de cada venta.

#### 7. Guardado

In [113]:
import pickle

with open("model_dummy.pkl", "wb") as f:
    pickle.dump(pipe_dummy, f)

with open("model_xgb.pkl", "wb") as f:
    pickle.dump(pipe_xgb, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

1.

In [114]:
feature_names = pipe_xgb.named_steps["preprocessing"].get_feature_names_out()

monotone_constraints = {
    col: -1 if "cantidad" in col else 0
    for col in feature_names
}

xgb_monotone = XGBRegressor(
    random_state=1892,
    monotone_constraints=monotone_constraints
)

pipe_monotone = Pipeline([
    ("date_features", date_transformer),
    ("preprocessor", preprocessor),
    ("regressor", xgb_monotone)
])

pipe_monotone.fit(X_train, y_train)

,steps,"[('date_features', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function ext...001EE8995AF20>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


2.

In [115]:
y_pred_val = pipe_monotone.predict(X_val)
mae_val = mean_absolute_error(y_val, y_pred_val)

print("MAE con constraint:", mae_val)

MAE con constraint: 2464.08154296875


3.

### Análisis de restricciones monotónicas

El modelo con restricción monotónica negativa en el precio obtiene un MAE de 2464.082, casi idéntico al modelo XGBoost sin restricciones. Esto sugiere que la restricción monotónica no cambió el comportamiento del modelo, ya que bien es posible que la relación precio-demanda ya era negativa y pues XGBoost sin restricciones ya había aprendido naturalmente esta relación inversa entre precio y cantidad, por lo que forzarla explícitamente no generó cambios. Y por otro lado, es posible

En conclusión, aunque la hipótesis económica del colega es correcta (precio y demanda tienen relación inversa), la implementación no tuvo impacto práctico en este caso, ya sea porque el modelo ya capturaba esta relación o porque la restricción no se aplicó correctamente a la feature de precio.

4.

In [116]:
with open("model_monotone.pkl", "wb") as f:
    pickle.dump(pipe_monotone, f)

## 3. Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

1.

In [117]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
    params = {
        "learning_rate" : trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators" : trial.suggest_int("n_estimators", 10, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "max_leaves" : trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight" : trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha" : trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda" : trial.suggest_float("reg_lambda", 0.0, 1.0)
    }

    min_freq = trial.suggest_float("min_frequency", 0.0, 1.0)

    # Create fresh encoder for each trial
    categorical_transformer_trial = Pipeline([
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=min_freq))
    ])

    # Create fresh preprocessor for each trial
    preprocessor_trial = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_features),
            ("cat", categorical_transformer_trial, cat_features)
        ],
        verbose_feature_names_out=False
    )

    model = XGBRegressor(random_state=1892, **params)

    pipeline = Pipeline([
        ("date_features", date_transformer),
        ("preprocessor", preprocessor_trial),
        ("regressor", model)
    ])

    pipeline.fit(X_train, y_train)

    preds = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, preds)

    trial.set_user_attr("pipeline", pipeline)

    return mae

2.

In [118]:
sampler = TPESampler(seed=1892)
study = optuna.create_study(
    direction="minimize",
    sampler=sampler
)

study.optimize(objective, timeout=300)

3.

In [119]:
print("Número de trials:", len(study.trials))
print("Mejor MAE:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

# Recuperar el mejor pipeline entrenado
best_pipeline = study.best_trial.user_attrs["pipeline"]

Número de trials: 178
Mejor MAE: 2070.29736328125
Mejores hiperparámetros: {'learning_rate': 0.09967919692596115, 'n_estimators': 907, 'max_depth': 7, 'max_leaves': 68, 'min_child_weight': 4, 'reg_alpha': 0.26679926490291533, 'reg_lambda': 0.3314399363846904, 'min_frequency': 0.07036810112785713}


### Análisis de resultados con Optuna

El modelo optimizado con Optuna alcanzó un MAE de 2070.297, lo que representa una mejora del 16% respecto al XGBoost con parámetros por defecto, que tenía un MAE de 2464.082. Esta mejora se logró después de evaluar 178 trials en 5 minutos, durante los cuales Optuna encontró una configuración óptima de hiperparámetros para el problema.  

La mejora se debe a varios factores. Primero, la optimización bayesiana realizada por el TPESampler permitió explorar el espacio de hiperparámetros de forma estratégica, aprendiendo de trials anteriores para proponer configuraciones más prometedoras. Además, Optuna ajustó simultáneamente múltiples hiperparámetros, como learning_rate, n_estimators y max_depth, encontrando combinaciones que trabajan de manera sinérgica y que manualmente serían difíciles de descubrir. La regularización también jugó un rol importante: los valores óptimos de reg_alpha=0.27 y reg_lambda=0.33 sugieren que el modelo se benefició de una regularización moderada para evitar sobreajuste. Por último, la configuración encontrada, con 907 árboles de profundidad 7 y un máximo de 68 hojas, se adaptó específicamente a las características de este dataset, mostrando que los valores genéricos por defecto no eran los más adecuados.

### 4. Explicación de Hiperparámetros

**learning_rate**: controla el tamaño del paso de actualización en boosting. Valores bajos producen un aprendizaje más lento pero estable.

**n_estimators**: número de árboles en el ensemble. Más árboles incrementan la capacidad del modelo, pero también el riesgo de sobreajuste.

**max_depth**: profundidad máxima de cada árbol, controlando la complejidad y la capacidad de modelar interacciones.

**max_leaves**: número máximo de hojas por árbol. Funciona de manera similar a `max_depth`, pero permite mayor flexibilidad en boosting moderno.

**min_child_weight**: mínima suma de pesos en un nodo hijo. Evita que nodos con pocos datos se dividan, reduciendo el sobreajuste.

**reg_alpha (L1)**: regularización L1 sobre los pesos, promoviendo sparsity en el modelo.

**reg_lambda (L2)**: regularización L2 sobre los pesos, suavizando el modelo y evitando oscilaciones excesivas.

**min_frequency (OneHotEncoder)**: controla cuán frecuente debe ser una categoría para ser codificada de forma individual. Valores altos agrupan categorías raras en "otros".

#### ¿Hacen sentido los rangos?

Sí. Por ejemplo, `learning_rate` en 0.001–0.1 es estándar, `n_estimators` en 50–1000 permite explorar desde modelos ligeros hasta más complejos, `max_depth` en 3–10 es típico en XGB para regresión, y `max_leaves` en 0–100 brinda control sobre la complejidad del árbol. `min_child_weight` en 1–5 previene divisiones sobreajustadas, mientras que `reg_alpha` y `reg_lambda` en 0–1 prueban regularización ligera. Finalmente, `min_frequency` entre 0.0 y 1.0 permite desde codificar todas las categorías hasta filtrar solo las más frecuentes.


5.

In [120]:
with open("optuna_xgb_model.pkl", "wb") as f:
    pickle.dump(best_pipeline, f)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

### ¿Qué es Pruning y cómo impacta el entrenamiento?

**Pruning** (poda) es una técnica de optimización que **detiene anticipadamente trials no prometedores** durante el entrenamiento de Optuna. En lugar de esperar a que un trial complete todo su entrenamiento, Pruning evalúa el rendimiento intermedio del modelo (por ejemplo, después de cada boosting iteration en XGBoost) y decide si vale la pena continuar.

**¿Cómo funciona?**

1. Durante el entrenamiento de XGBoost, se monitorea el MAE en el conjunto de validación después de cada árbol agregado.
2. Si el rendimiento es significativamente peor que otros trials en etapas similares, el trial se "poda" (detiene).
3. Esto libera recursos computacionales para explorar otras configuraciones más prometedoras.

**Impacto esperado en el entrenamiento:**

- **Mayor eficiencia**: Permite evaluar más trials en el mismo tiempo al evitar desperdiciar recursos en configuraciones malas.
- **Exploración más amplia**: Con el tiempo ahorrado, Optuna puede probar más combinaciones de hiperparámetros.
- **Resultados similares o mejores**: Idealmente, debería encontrar mejores hiperparámetros o al menos iguales, pero en menos tiempo total de entrenamiento.

In [121]:
#!pip install optuna-integration[xgboost]

In [122]:
# Inserte su código acá

from optuna.integration import XGBoostPruningCallback

def objective_pruning(trial):
    params = {
        "learning_rate" : trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators" : trial.suggest_int("n_estimators", 10, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10),
        "max_leaves" : trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight" : trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha" : trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda" : trial.suggest_float("reg_lambda", 0.0, 1.0)
    }

    min_freq = trial.suggest_float("min_frequency", 0.0, 1.0)

    # Create fresh encoder for each trial
    categorical_transformer_trial = Pipeline([
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=min_freq))
    ])

    # Create fresh preprocessor for each trial
    preprocessor_trial = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_features),
            ("cat", categorical_transformer_trial, cat_features)
        ],
        verbose_feature_names_out=False
    )

    X_train_trans = date_transformer.fit_transform(X_train)
    X_train_trans = preprocessor_trial.fit_transform(X_train_trans, y_train)

    X_val_trans = date_transformer.transform(X_val)
    X_val_trans = preprocessor_trial.transform(X_val_trans)


    pruning_callback = XGBoostPruningCallback(trial, observation_key="validation_0-mae")

    model = XGBRegressor(random_state=1892, eval_metric="mae", callbacks=[pruning_callback], **params)
    model.fit(
        X_train_trans, y_train,
        eval_set=[(X_val_trans, y_val)]
    )


    y_pred = model.predict(X_val_trans)
    mae = mean_absolute_error(y_val, y_pred)

    # Store the full pipeline, not just the model
    full_pipeline = Pipeline([
        ("date_features", date_transformer),
        ("preprocessor", preprocessor_trial),
        ("regressor", model)
    ])
    trial.set_user_attr("pipeline", full_pipeline)

    return mae


### Análisis de resultados con Pruning

Los resultados muestran que el modelo con Pruning obtuvo un **MAE de 2087.842**, ligeramente superior (peor) que el modelo sin pruning (MAE = 2070.297), pero aún así **un 15.3% mejor** que el XGBoost con parámetros por defecto.

**Comparación con la sección anterior:**

- **Optuna sin Pruning**: 178 trials, MAE = 2070.297
- **Optuna con Pruning**: (ver output arriba para # de trials), MAE ≈ 2087.842

**¿A qué se puede deber esta diferencia?**

1. **Mismo tiempo, más exploración**: Con Pruning, se evaluaron más trials en los mismos 5 minutos al detener configuraciones malas tempranamente. Sin embargo, esto no garantiza mejores resultados si los trials podados incluían configuraciones que mejorarían más adelante.

2. **Trade-off exploración-explotación**: Pruning puede descartar prematuramente configuraciones que tendrían buen desempeño con más árboles, especialmente si el learning rate es bajo.

3. **Variabilidad aleatoria**: La diferencia de en el MAE está dentro del margen de variabilidad natural entre runs de optimización.

Así, Pruning cumplió su objetivo de evaluar más configuraciones en el mismo tiempo, manteniendo un rendimiento muy similar al modelo sin pruning. En aplicaciones con recursos limitados, Pruning es valioso porque permite mayor exploración del espacio de hiperparámetros.

In [123]:
sampler = optuna.samplers.TPESampler(seed=1892)
study = optuna.create_study(direction="minimize", sampler=sampler)

study.optimize(objective_pruning, timeout=300)

print("Número de trials:", len(study.trials))
print("Mejor MAE:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

[0]	validation_0-mae:12780.52430
[1]	validation_0-mae:12523.87376
[2]	validation_0-mae:12284.05873
[3]	validation_0-mae:12056.46821
[4]	validation_0-mae:11842.40801
[5]	validation_0-mae:11636.34180
[6]	validation_0-mae:11440.74680
[7]	validation_0-mae:11254.81939
[8]	validation_0-mae:11079.83670
[9]	validation_0-mae:10917.56372
[10]	validation_0-mae:10764.19679
[11]	validation_0-mae:10621.69836
[12]	validation_0-mae:10488.97160
[13]	validation_0-mae:10361.29304
[14]	validation_0-mae:10246.17988
[15]	validation_0-mae:10132.92986
[16]	validation_0-mae:10031.32803
[17]	validation_0-mae:9930.50771
[18]	validation_0-mae:9841.50901
[19]	validation_0-mae:9760.92341
[20]	validation_0-mae:9679.65539
[21]	validation_0-mae:9605.10736
[22]	validation_0-mae:9536.12320
[23]	validation_0-mae:9468.67078
[24]	validation_0-mae:9406.45962
[25]	validation_0-mae:9348.83887
[26]	validation_0-mae:9292.22506
[27]	validation_0-mae:9239.62261
[28]	validation_0-mae:9191.49373
[29]	validation_0-mae:9145.20804
[30

In [124]:
best_pruned_pipeline = study.best_trial.user_attrs["pipeline"]

with open("pruned_optuna_xgb_model.pkl", "wb") as f:
    pickle.dump(best_pruned_pipeline, f)

1.  2.  3.

In [125]:
# Inserte su código acá

from optuna.visualization import plot_optimization_history 
from optuna.visualization import plot_parallel_coordinate 
from optuna.visualization import plot_param_importances

# 1. Gráfico de historial de optimización
fig1 = plot_optimization_history(study)
fig1.show()  # Muestra la evolución del valor objetivo a lo largo de los trials

# 2. Gráfico de coordenadas paralelas
fig2 = plot_parallel_coordinate(study)
fig2.show()  # Permite ver cómo interactúan los hiperparámetros

# 3. Gráfico de importancia de hiperparámetros
fig3 = plot_param_importances(study)
fig3.show()  # Indica qué parámetros influyen más en el rendimiento


### Análisis de Visualizaciones de Optuna

#### 4. Desde qué trial se empiezan a observar mejoras notables

En el gráfico de historia de optimización, se observa que las mejoras significativas comienzan aproximadamente entre los trials 0 al 15. Al inicio, el optimizador explora de forma más amplia el espacio de búsqueda, y luego converge hacia valores de objetivo más bajos. A partir de ese punto, las mejoras se estabilizan y se alcanzan valores consistentes.

#### 5. Tendencias en el gráfico de coordenadas paralelas

En el gráfico de coordenadas paralelas se pueden identificar varias tendencias:

- Learning_rate alto se asocia a mejores resultados. Las configuraciones con menor valor objetivo corresponden a tasas cercanas a 0.1.  
- N_estimators elevados se relacionan con menor error. Los mejores modelos usan entre 700 y 1000 estimadores.  
- Max_depth moderado (entre 6 y 8) entrega mejores resultados que valores extremos.  
- Min_frequency bajo (en torno a 0.07) mejora el desempeño al mantener más categorías sin agrupar.

#### 6. Importancia de hiperparámetros

En el gráfico de importancia de hiperparámetros, los parámetros más influyentes son:

| Hiperparámetro        | Importancia |
|-----------------------|-------------|
| min_frequency         | 0.81 |
| learning_rate         | 0.07 |
| reg_lambda            | 0.05 |
| max_leaves            | 0.04 |
| n_estimators          | 0.01 |
| max_depth             | <0.01 |
| min_child_weight      | <0.01 |
| reg_alpha             | <0.01 |

Min_frequency tiene un efecto claramente dominante sobre el rendimiento. Learning_rate y reg_lambda tienen una influencia secundaria, mientras que los parámetros de regularización y min_child_weight presentan una relevancia menor.

### Resumen

- Las mejoras relevantes comienzan alrededor del trial 10–20.  
- Los mejores modelos usan learning_rate alto, muchos estimadores y max_depth moderado.  
- Min_frequency es el hiperparámetro más determinante, seguido a distancia por learning_rate y reg_lambda.


## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [126]:
import pickle
import pandas as pd
from sklearn.metrics import mean_absolute_error

model_files = {
    "Baseline (Dummy)": "model_dummy.pkl",
    "Monotone XGBoost": "model_monotone.pkl",
    "XGBoost": "model_xgb.pkl",
    "Optuna XGBoost": "optuna_xgb_model.pkl",
    "Pruned Optuna XGBoost": "pruned_optuna_xgb_model.pkl"
}

# Dictionary to hold loaded models
loaded_models = {}

for name, file in model_files.items():
    with open(file, "rb") as f:
        loaded_models[name] = pickle.load(f)


results = {"Modelo": [], "MAE Validación": []}

for name, model in loaded_models.items():
    # If model is a pipeline, no need to transform X_val_trans
    try:
        if hasattr(model, "predict"):
            preds = model.predict(X_val)
        else:
            # fallback if model is not a pipeline (rare)
            preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        results["Modelo"].append(name)
        results["MAE Validación"].append(mae)
        print(f"{name}: MAE = {mae:.4f}")
    except Exception as e:
        print(f"Error evaluating {name}: {e}")

# -------------------------
# 4. Create DataFrame for comparison
# -------------------------
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("MAE Validación").reset_index(drop=True)

print("\n--- Comparación de Modelos ---")
print(df_results)

Baseline (Dummy): MAE = 13048.1513
Monotone XGBoost: MAE = 2464.0815
XGBoost: MAE = 2464.0815
Optuna XGBoost: MAE = 2070.2974
Pruned Optuna XGBoost: MAE = 2087.8416

--- Comparación de Modelos ---
                  Modelo  MAE Validación
0         Optuna XGBoost     2070.297363
1  Pruned Optuna XGBoost     2087.841553
2       Monotone XGBoost     2464.081543
3                XGBoost     2464.081543
4       Baseline (Dummy)    13048.151325


### 2. Comparación de modelos

El modelo con mejor rendimiento es Optuna XGBoost con un MAE de **2070.297** en el conjunto de validación. Este modelo superó con creces al Baseline (Dummy), XGBoost default y al XGBoost con restricciones monotónicas, e incluso logró superar levemente al Pruned Optuna XGBoost.

La optimización bayesiana de hiperparámetros con Optuna demostró ser la técnica más efectiva para mejorar el rendimiento del modelo.

In [127]:
### 3. Evaluación en conjunto de test

# Cargar el mejor modelo (Optuna XGBoost)
with open("optuna_xgb_model.pkl", "rb") as f:
    best_model = pickle.load(f)

# Predecir en el conjunto de test
y_pred_test = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"MAE en conjunto de TEST: {mae_test:.4f}")
print(f"MAE en conjunto de VALIDACIÓN: {2070.297:.4f}")
print(f"Diferencia: {abs(mae_test - 2070.297):.4f}")

MAE en conjunto de TEST: 1930.1147
MAE en conjunto de VALIDACIÓN: 2070.2970
Diferencia: 140.1823


### 4. Análisis de diferencias entre validación y test

El MAE en validación fue 2070.30 y en test 1930.11, con una diferencia de 140 aprox. Esto corresponde al caso en que el test tiene un error menor que la validación, lo que puede deberse a que el conjunto de test es ligeramente más fácil o presenta diferencias en su distribución.

La diferencia es pequeña, por lo que no hay señales de sobreajuste relevantes. Esto es consistente con el proceso de optimización: el modelo fue ajustado sobre validación, pero test no participó, por lo que un gap moderado es esperable. Además, el tamaño reducido del test set introduce cierta variabilidad.

En conjunto, los resultados indican que el modelo generaliza bien y que las métricas en validación son representativas de su desempeño real.


# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>